In [13]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras import backend as K
#from theano import tensor as K

import numpy as np
import os
import PIL
from PIL import Image

In [14]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64,3,3,activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [15]:
#model = VGG_16('vgg16_weights_tf_dim_ordering_tf_kernels.h5')
model = VGG_16('vgg16_weights_th_dim_ordering_th_kernels.h5')
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy')

C:\Users\hahmed\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
C:\Users\hahmed\Anaconda2\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
C:\Users\hahmed\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\hahmed\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\Users\hahmed\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Us

In [16]:
size = 224
def read_image(f):
    im = Image.open(f)
    im = im.resize((size, size), PIL.Image.NEAREST)
    im = np.asarray(im, dtype='float64')
    im[:,:,0] -= 103.939
    im[:,:,1] -= 116.779
    im[:,:,2] -= 123.68
    im = im.transpose((2,0,1))
    im = np.expand_dims(im, axis=0)
    im = im.reshape(1,3,224,224)
    
    return(im)
#resize has multiple algorithms, one of them is nearest
#np.asarray is used to convert image from PIL image format to array format
#103,116,123 are rescaling  / inverse scaling parameters - these are means from VGG channel means ; 
#VGG has trained using scaled input data with above means 

In [17]:
im = read_image('C:\\MOOC\\Insofe\\Module 5- Advance Machine Learning\\20170916_Batch 29_CSE7321c_Transfer_Learning_LabDay\\20170916_Batch 29_CSE7321c_Transfer_Learning\\test_samples\\1.jpg')
print(im.shape)
model.layers[32]

(1L, 3L, 224L, 224L)


In [18]:
# with a Sequential model
get_33rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[32].output])
layer_output = get_33rd_layer_output([im])[0]
print(layer_output.shape)

#the backend function in keras which is K.function uses our input and VGG net architecture layers
#the 33 model layer output is given for our input
#we got 4096 features which can be used as input to your own mlp

(1L, 4096L)


In [19]:
img_files = []
for root, dirs, files in os.walk('C:\\MOOC\\Insofe\\Module 5- Advance Machine Learning\\20170916_Batch 29_CSE7321c_Transfer_Learning_LabDay\\20170916_Batch 29_CSE7321c_Transfer_Learning\\train_samples'):
    img_files.extend(files)

In [9]:
n_files = len(img_files)
print('Total number of images:', n_files)

#train_size = 2000
#for demo
train_size = 10
sample_size = max(n_files,train_size)
print('Sample images taken:', sample_size)
sample = [img_files[i] for i in np.random.choice(len(files),size=sample_size, replace=False)]

i=0
x_train = []
y_train = []
print('Reading train images ...')
for file in sample:
    if(i%200==0):
        print('Read {} images'.format(i))
    im = read_image('C:\\MOOC\\Insofe\\Module 5- Advance Machine Learning\\20170916_Batch 29_CSE7321c_Transfer_Learning_LabDay\\20170916_Batch 29_CSE7321c_Transfer_Learning\\train_samples/'+file)
    temp = get_33rd_layer_output([im])[0]
    x_train.append(temp[0])
    y_train.extend([0 if file[:3]=='cat' else 1])
    i+=1

x_train = np.array(x_train)
print(x_train.shape, 'x_train Shape')

y_train = to_categorical(y_train)
print(y_train.shape, 'y_train Shape')

('Total number of images:', 4000)
('Sample images taken:', 4000)
Reading train images ...
Read 0 images
((1L, 4096L), 'x_train Shape')
((1L, 2L), 'y_train Shape')


In [10]:
mlp = Sequential()
mlp.add(Dense(256, input_dim=4096, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(128, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(2, activation='softmax'))

#adam = Adam(lr=0.1)
mlp.compile(optimizer='Adam', 
            loss='categorical_crossentropy', 
            metrics=['acc'])

In [12]:
mlp.fit(x_train, y_train, epochs=10)

Epoch 1/1
1/1 [==============================] - 0s - loss: 14.9810 - acc: 0.0000e+00


In [58]:
model.layers